# [[데이콘] 영화 리뷰 감성 분석 PBL2](https://dacon.io/competitions/official/235995/codeshare/6529?page=1&dtype=recent)

In [ ]:
!pip install konlpy

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import re
import sys
import nltk
from collections import Counter
from wordcloud import WordCloud
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
nltk.download('all')

In [2]:
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/train.csv')[['document', 'label']]
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/test.csv')[['document']]
submission = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/sample_submission.csv')

In [ ]:
train

In [ ]:
test

In [ ]:
# train/test 결측치 확인
print('train 결측치 개수\n',train.isnull().sum())
print(train[train['document'].isnull()])
print('\n')
print('test 결측치 개수\n',test.isnull().sum())
print(test[test['document'].isnull()])

In [6]:
# train/test 결측치 대체
train = train.fillna(method='ffill')
test = test.fillna(method='ffill')

In [ ]:
# 결측치 개수 확인
print(train.isnull().sum())
print(test.isnull().sum())

### 1-3. Feature & Target 설정

In [8]:
train_feature = train.document
train_label = train.label

In [9]:
train_feature.head()

0                                  아 더빙.. 진짜 짜증나네요 목소리
1                    흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
2                                    너무재밓었다그래서보는것을추천한다
3                        교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정
4    사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...
Name: document, dtype: object

In [ ]:
train_feature.tail()

## 2. 텍스트 전처리
### 2-1. CountVectorizer의 이해
- 워드 임베딩 진행
  - 워드 임베딩: 컴퓨터가 계산하기 위해 문장을 숫자로 바꾸는 것(단어를 벡터로 표현)
- 본 분석에서는 CountVectorizer를 이용해 문장을 숫자로 표현
  - CountVectorizer: 입력된 문장을 **토큰화**하여 토큰의 **등장 빈도 벡터**로 바꾸는 기법


In [11]:
# 예시
from sklearn.feature_extraction.text import CountVectorizer

sample_vectorizer = CountVectorizer()
sample_text1 = ["hello, my name is dacon and I am a data scientist!"]
sample_vectorizer.fit(sample_text1) # CountVectorizer 학습

print(sample_vectorizer.vocabulary_) # sample_vectorizer가 학습한 단어 목록

{'hello': 4, 'my': 6, 'name': 7, 'is': 5, 'dacon': 2, 'and': 1, 'am': 0, 'data': 3, 'scientist': 8}


길이가 1개인 단어들과 특수문자는 의미가 적다고 판단하여 제외되었음

In [12]:
# 다른 문장을 활용해 벡터로 transform 진행
sample_text2 = ["you are learning dacon data science"]

sample_vector = sample_vectorizer.transform(sample_text2)
print(sample_vector.toarray())

[[0 0 1 1 0 0 0 0 0]]


↑ transform의 결과, 단어들의 출현 빈도로 이루어진 크기 9의 벡터 출력됨

이 벡터를 BoW라고 하는데, BoW란 CountVectorizer로 변환된 단어의 집합을 의미함

sample_text2에는 dacon과 data가 1번씩 출현했기에 해당 인덱스 값이 1로 표현되었고, 등장하지 않은 단어들은 0으로 표시됨

In [13]:
sample_text3 = ["you are learning dacon data science with movie data"]

sample_vector = sample_vectorizer.transform(sample_text3)
print(sample_vector.toarray())

[[0 0 1 2 0 0 0 0 0]]


"dacon" 단어가 1번, "data" 단어가 2번 출현했기에 "dacon" 단어에 해당하는 인덱스에는 1의 값이, "data" 에 해당하는 인덱스에는 2의 값이 할당됨

CountVectorizer 를 하나의 문장을 사용해 학습 시켰기에 생성된 Vocab과 BoW 의 크기는 모두 9였음

큰 데이터를 사용해 CountVectorizer를 학습시키면 Vocab과 BoW의 크기는 증가할 것

In [14]:
vectorizer = CountVectorizer() # countvectorizer 생성
vectorizer.fit(train_feature) # countvectorizer 학습(fit)
train_X = vectorizer.transform(train_feature) # transform

In [15]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(train_X, train_label)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

위에서 train 데이터로 학습한 모델로 test data를 넣어 예측값을 도출함

fit_transform() 매서드는 학습을 위한 train data에 사용되고 test data에는 transform() 매서드만 사용해야 함

test data에도 fit_transform() 매서드를 사용하는 것은 모델이 test data에 대해서도 학습을 한다는 것을 의미함 → No!!

In [16]:
test_X_vect = vectorizer.transform(test['document']) # test data는 transform() 매서드만 적용

pred = model.predict(test_X_vect)
print(pred)

[0 0 0 ... 1 0 0]


### 2-2. 데이터 정제


In [18]:
from sklearn.model_selection import train_test_split

train, val = train_test_split(train)

# 전처리 과정에서 데이터가 뒤섞이지 않도록 인덱스를 초기화
train.reset_index(inplace=True)
test.reset_index(inplace=True)

In [20]:
train.head()

,index,document,label
0,138275,감동적입니다. 저런 따뜻한 선생님들만 있으면 우리나라 엄청 발전하겠죠,1
1,148248,왠 영화난지도... 이 소모품에 자신의 이름이 들어 갔다는 것이 평생을 후회할 듯....,0
2,4033,똥파스틱. . . . 후. . .,0
3,38107,재미만 있구만ㅋ 근데 전설이라는 것들이 비교적 약해보임,1
4,84018,최고 재밌네요 밑에 다들 경쟁작 알바에요꼭 보세요!!!두번보세요!!!이렇게 재밌는 ...,1


In [23]:
val.head()

,document,label
116785,엄......,0
129766,마이너스 별은 없녜 ㅋㅋㅋㅋㅋㅋㅋ 진짜 인내심 필요한 영화네요 ㅋㅋㅋ,0
90422,여운이 남는다 ....,1
77639,요즘들어 질질 끄는게 보여서 재미가업어여...,0
24621,만점까진아니고.. 일단 이건 개봉후부터 지금까지 7번인가봤는데 볼때마다 새로운거같음...,1


In [28]:
train['preprocessed'] = train['document'].str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]', '') # 한글과 공백을 제외하고 모두 제거
train[:5]

<ipython-input-28-f2548bcc9f40>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  train['preprocessed'] = train['document'].str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]', '') # 한글과 공백을 제외하고 모두 제거


,index,document,label,preprocessed
0,138275,감동적입니다. 저런 따뜻한 선생님들만 있으면 우리나라 엄청 발전하겠죠,1,감동적입니다 저런 따뜻한 선생님들만 있으면 우리나라 엄청 발전하겠죠
1,148248,왠 영화난지도... 이 소모품에 자신의 이름이 들어 갔다는 것이 평생을 후회할 듯....,0,왠 영화난지도 이 소모품에 자신의 이름이 들어 갔다는 것이 평생을 후회할 듯 별의 ...
2,4033,똥파스틱. . . . 후. . .,0,똥파스틱 후
3,38107,재미만 있구만ㅋ 근데 전설이라는 것들이 비교적 약해보임,1,재미만 있구만ㅋ 근데 전설이라는 것들이 비교적 약해보임
4,84018,최고 재밌네요 밑에 다들 경쟁작 알바에요꼭 보세요!!!두번보세요!!!이렇게 재밌는 ...,1,최고 재밌네요 밑에 다들 경쟁작 알바에요꼭 보세요두번보세요이렇게 재밌는 영화 놓치시...


In [29]:
train['preprocessed'] = train['preprocessed'].str.replace(' +', ' ') # 다중 공백 제거
train[:5]

<ipython-input-29-382d39dcefb0>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  train['preprocessed'] = train['preprocessed'].str.replace(' +', ' ') # 다중 공백 제거


,index,document,label,preprocessed
0,138275,감동적입니다. 저런 따뜻한 선생님들만 있으면 우리나라 엄청 발전하겠죠,1,감동적입니다 저런 따뜻한 선생님들만 있으면 우리나라 엄청 발전하겠죠
1,148248,왠 영화난지도... 이 소모품에 자신의 이름이 들어 갔다는 것이 평생을 후회할 듯....,0,왠 영화난지도 이 소모품에 자신의 이름이 들어 갔다는 것이 평생을 후회할 듯 별의 ...
2,4033,똥파스틱. . . . 후. . .,0,똥파스틱 후
3,38107,재미만 있구만ㅋ 근데 전설이라는 것들이 비교적 약해보임,1,재미만 있구만ㅋ 근데 전설이라는 것들이 비교적 약해보임
4,84018,최고 재밌네요 밑에 다들 경쟁작 알바에요꼭 보세요!!!두번보세요!!!이렇게 재밌는 ...,1,최고 재밌네요 밑에 다들 경쟁작 알바에요꼭 보세요두번보세요이렇게 재밌는 영화 놓치시...


### 2-3. 토큰화
- 토큰화(Tokenization): 말뭉치(Corpus)를 주어진 단위(Token)로 나누는 과정
  1. 문장 토큰화
  1. 단어 토큰화

In [33]:
text = "Hello, nice to meet you. What's your name? Have a nice day! See you soon." 

from nltk.tokenize import sent_tokenize

print('문장 토큰화 결과 ==>',sent_tokenize(text))

from nltk.tokenize import word_tokenize

print('단어 토큰화 결과 ==>', word_tokenize(text))

문장 토큰화 결과 ==> ['Hello, nice to meet you.', "What's your name?", 'Have a nice day!', 'See you soon.']
단어 토큰화 결과 ==> ['Hello', ',', 'nice', 'to', 'meet', 'you', '.', 'What', "'s", 'your', 'name', '?', 'Have', 'a', 'nice', 'day', '!', 'See', 'you', 'soon', '.']


In [34]:
from konlpy.tag import Okt 

han_sentence = "오늘도 열심히 코딩을 해볼까요? 같이 힘내서 자연어 처리 고수가 됩시다! ㅎㅎ"
okt = Okt() # 인스턴스 할당
print("한국어 형태소 분석 결과(어간 추출X) ==>", okt.morphs(han_sentence, stem = False)) # 형태소 단위로 분리
print("한국어 형태소 분석 결과(어간 추출O) ==>", okt.morphs(han_sentence, stem = True)) # 형태소 단위로 분리 후 어간 추출

한국어 형태소 분석 결과(어간 추출X) ==> ['오늘', '도', '열심히', '코딩', '을', '해볼까', '요', '?', '같이', '힘내서', '자연어', '처리', '고수', '가', '됩시다', '!', 'ㅎㅎ']
한국어 형태소 분석 결과(어간 추출O) ==> ['오늘', '도', '열심히', '코딩', '을', '해보다', '요', '?', '같이', '힘내다', '자연어', '처리', '고수', '가', '되다', '!', 'ㅎㅎ']


In [35]:
tokenized = [] # 데이터프레임의 한 칼럼의 추가할 리스트
for sentence in train['preprocessed']: # 전처리된 리뷰들을 하나씩 꺼내어기
  tokens = okt.morphs(sentence, stem=True) # 형태소 분석(stem=True로 어간 추출 진행)
  tokenize = " ".join(tokens) # tokens라는 리스트 안에 형태소들을 띄어쓰기로 분리된 하나의 문자열로 join
  tokenized.append(tokenize) # 형태소 단위로 띄어쓰기된 문자열을 최종 리스트에 추가
train['tokenized_stem'] = pd.DataFrame(tokenized) # 리스트를 데이터프레임으로 변환해 tokenized_stem라는 칼럼명으로 추가

train.head() # 데이터 확인

,index,document,label,preprocessed,tokenized_stem
0,138275,감동적입니다. 저런 따뜻한 선생님들만 있으면 우리나라 엄청 발전하겠죠,1,감동적입니다 저런 따뜻한 선생님들만 있으면 우리나라 엄청 발전하겠죠,감동 적 이다 저런 따뜻하다 선생님 들 만 있다 우리나라 엄청 발전 하다
1,148248,왠 영화난지도... 이 소모품에 자신의 이름이 들어 갔다는 것이 평생을 후회할 듯....,0,왠 영화난지도 이 소모품에 자신의 이름이 들어 갔다는 것이 평생을 후회할 듯 별의 ...,왠 영화 난지도 이 소모품 에 자신 의 이름 이 들다 가다 것 이 평생 을 후회 하...
2,4033,똥파스틱. . . . 후. . .,0,똥파스틱 후,똥 파스 틱 후
3,38107,재미만 있구만ㅋ 근데 전설이라는 것들이 비교적 약해보임,1,재미만 있구만ㅋ 근데 전설이라는 것들이 비교적 약해보임,재미 만 있다 ㅋ 근데 전설 이라는 것 들 이 비교 적 약하다 보임
4,84018,최고 재밌네요 밑에 다들 경쟁작 알바에요꼭 보세요!!!두번보세요!!!이렇게 재밌는 ...,1,최고 재밌네요 밑에 다들 경쟁작 알바에요꼭 보세요두번보세요이렇게 재밌는 영화 놓치시...,최고 재밌다 밑 에 다 들다 경쟁 작 알바 에요 꼭 보다 두 번 보다 이렇게 재밌다...


### 2-4. 품사 태깅(POP Tagging)
- 품사 태깅: 주어진 텍스트를 형태소 단위로 나눈 뒤, 각 형태소에 해당 품사를 태깅하여 리스트화 하는 것

In [36]:
# 품사 태깅
print(okt.pos("오늘도 열심히 재밌는 코딩을 해볼까? 같이 힘내서 자연어 처리 고수들이 됩시다! ㅎㅎ"))

[('오늘', 'Noun'), ('도', 'Josa'), ('열심히', 'Adverb'), ('재밌는', 'Adjective'), ('코딩', 'Noun'), ('을', 'Josa'), ('해볼까', 'Verb'), ('?', 'Punctuation'), ('같이', 'Adverb'), ('힘내서', 'Verb'), ('자연어', 'Noun'), ('처리', 'Noun'), ('고수', 'Noun'), ('들', 'Suffix'), ('이', 'Josa'), ('됩시다', 'Verb'), ('!', 'Punctuation'), ('ㅎㅎ', 'KoreanParticle')]


In [37]:
print(okt.pos("이것은 1점이 아니다 11점을 주고싶은 내 간절한 마음이다"))

[('이', 'Determiner'), ('것', 'Noun'), ('은', 'Josa'), ('1', 'Number'), ('점', 'Noun'), ('이', 'Josa'), ('아니다', 'Adjective'), ('11', 'Number'), ('점', 'Noun'), ('을', 'Josa'), ('주고싶은', 'Verb'), ('내', 'Noun'), ('간절한', 'Adjective'), ('마음', 'Noun'), ('이다', 'Josa')]


In [38]:
# 명사 추출
print(okt.nouns("오늘도 열심히 재밌는 코딩을 해볼까? 같이 힘내서 자연어 처리 고수들이 됩시다! ㅎㅎ"))

['오늘', '코딩', '자연어', '처리', '고수']


In [40]:
main_pos = []
for sentence in train['document']:
  pos = okt.pos(sentence)
  main_words = [word_pos[0] for word_pos in pos if word_pos[1] in ("Noun", "Adverb", "Adjective", "Verb")]
  main_words_str = " ".join(main_words)
  main_pos.append(main_words_str)
train["main_pos"] = pd.DataFrame(main_pos)

train.head() 

,index,document,label,preprocessed,tokenized_stem,main_pos
0,138275,감동적입니다. 저런 따뜻한 선생님들만 있으면 우리나라 엄청 발전하겠죠,1,감동적입니다 저런 따뜻한 선생님들만 있으면 우리나라 엄청 발전하겠죠,감동 적 이다 저런 따뜻하다 선생님 들 만 있다 우리나라 엄청 발전 하다,감동 입니다 저런 따뜻한 선생님 있으면 우리나라 엄청 발전 하겠죠
1,148248,왠 영화난지도... 이 소모품에 자신의 이름이 들어 갔다는 것이 평생을 후회할 듯....,0,왠 영화난지도 이 소모품에 자신의 이름이 들어 갔다는 것이 평생을 후회할 듯 별의 ...,왠 영화 난지도 이 소모품 에 자신 의 이름 이 들다 가다 것 이 평생 을 후회 하...,영화 난지도 이 소모품 자신 이름 들어 갔다는 것 평생 후회 할 듯 별 반쪽 너무 ...
2,4033,똥파스틱. . . . 후. . .,0,똥파스틱 후,똥 파스 틱 후,똥 파스 후
3,38107,재미만 있구만ㅋ 근데 전설이라는 것들이 비교적 약해보임,1,재미만 있구만ㅋ 근데 전설이라는 것들이 비교적 약해보임,재미 만 있다 ㅋ 근데 전설 이라는 것 들 이 비교 적 약하다 보임,재미 있구만 근데 전설 것 비교 약해 보임
4,84018,최고 재밌네요 밑에 다들 경쟁작 알바에요꼭 보세요!!!두번보세요!!!이렇게 재밌는 ...,1,최고 재밌네요 밑에 다들 경쟁작 알바에요꼭 보세요두번보세요이렇게 재밌는 영화 놓치시...,최고 재밌다 밑 에 다 들다 경쟁 작 알바 에요 꼭 보다 두 번 보다 이렇게 재밌다...,최고 재밌네요 밑 다 들 경쟁 작 알바 꼭 보세요 번 보세요 이렇게 재밌는 영화 놓...


마지막에 생성된 main_pos 칼럼을 이용해 모델 학습

## 3. 모델링

In [41]:
X_train = train.main_pos
y_train = train.label

### 3.1 검증 데이터셋 전처리
먼저 검증 셋 val에 train 셋과 동일한 전처리 과정을 거쳐줍니다.

In [47]:
val['preprocessed'] = val['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "")
val['preprocessed'] = val['preprocessed'].str.replace("[ +]", " ")

tokenized = []
for sentence in val['preprocessed']:
  tokens = okt.morphs(sentence)
  tokenize = " ".join(tokens)
  tokenized.append(tokenize)
val['tokenized_stem'] = pd.DataFrame(tokenized)

main_pos = []
for sentence in val['document']:
  pos = okt.pos(sentence)
  main_words = [word_pos[0] for word_pos in pos if word_pos[1] in ("Noun", "Adverb", "Adjective", "Verb")]
  main_words_str = " ".join(main_words)
  main_pos.append(main_words_str)
val["main_pos"] = pd.DataFrame(main_pos)
val.head()

<ipython-input-47-b5bf1a72a988>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  val['preprocessed'] = val['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "")
<ipython-input-47-b5bf1a72a988>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  val['preprocessed'] = val['preprocessed'].str.replace("[ +]", " ")


,document,label,preprocessed,tokenized_stem,main_pos
116785,엄......,0,엄,NaN,NaN
129766,마이너스 별은 없녜 ㅋㅋㅋㅋㅋㅋㅋ 진짜 인내심 필요한 영화네요 ㅋㅋㅋ,0,마이너스 별은 없녜 ㅋㅋㅋㅋㅋㅋㅋ 진짜 인내심 필요한 영화네요 ㅋㅋㅋ,NaN,NaN
90422,여운이 남는다 ....,1,여운이 남는다,NaN,NaN
77639,요즘들어 질질 끄는게 보여서 재미가업어여...,0,요즘들어 질질 끄는게 보여서 재미가업어여,NaN,NaN
24621,만점까진아니고.. 일단 이건 개봉후부터 지금까지 7번인가봤는데 볼때마다 새로운거같음...,1,만점까진아니고 일단 이건 개봉후부터 지금까지 번인가봤는데 볼때마다 새로운거같음 연기...,진짜 재밌게 봄 ㅋㅋ,진짜 재밌게 봄


### 3-2. LogisticRegression 모델 정의

In [49]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer() # countvectorizer 생성
vectorizer.fit(X_train) # countvectorizer 학습
X_train_vec = vectorizer.transform(X_train) # transform

# 3-3. 모델 학습

In [50]:
from sklearn.linear_model import LogisticRegression #모델 불러오기

model = LogisticRegression()
model.fit(X_train_vec, y_train) # 모델 학습

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [52]:
# 학습한 모델로 에측할 X를 벡터화 진행
X_val = val.main_pos
y_val = val.label

X_val_vec = vectorizer.transform(X_val) # train셋으로 fit한 벡터라이저 이용해 transform

ValueError: ignored

### 3-4. 모델 예측

In [53]:
y_pred = model.predict(X_val_vec)
print(y_pred)

# 0:negative 1:positive

NameError: ignored

### 3-5. 검증

In [ ]:
from sklearn import metrics
print('accuracy =', metrics.accuracy_score(y_val, y_pred)) # 정확도 확인

## 4. 제출